# Data Loading

We are provided with a set of CSV files. Let's load them into an SQL repository to conduct some preliminary data exploration.

In [1]:
import duckdb
import pandas as pd
from pathlib import Path

In [2]:
# Connect to DuckDB (creates file if doesn't exist)
con = duckdb.connect("data/aida_challenge.duckdb")

In [7]:
# Define data sources
data_files = {
    "clienti": "data/raw/clienti.csv",
    "polizze": "data/raw/polizze.csv",
    "sinistri": "data/raw/sinistri.csv",
    "reclami": "data/raw/reclami.csv",
    "abitazioni": "data/raw/abitazioni.csv",
    "interazioni_clienti": "data/raw/interazioni_clienti.csv",
    "competitor_prodotti": "data/raw/competitor_prodotti.csv",
}

# Load each CSV into DuckDB
for table_name, file_path in data_files.items():
    con.execute(
        f"""
        CREATE TABLE IF NOT EXISTS {table_name} AS 
        SELECT * FROM read_csv_auto('{file_path}')
    """
    )
    print(f"✓ Loaded {table_name}")

# Quick verification
print("\nTables in database:")
print(con.execute("SHOW TABLES").df())

✓ Loaded clienti
✓ Loaded polizze
✓ Loaded sinistri
✓ Loaded reclami
✓ Loaded abitazioni
✓ Loaded interazioni_clienti
✓ Loaded competitor_prodotti

Tables in database:
                  name
0           abitazioni
1              clienti
2  competitor_prodotti
3  interazioni_clienti
4              polizze
5              reclami
6             sinistri


In [8]:
# Check schema and sample data for each table
for table_name in data_files.keys():
    print(f"\n{'='*60}")
    print(f"TABLE: {table_name}")
    print(f"{'='*60}")

    # Show data types and column info
    print("\nSchema:")
    schema = con.execute(f"DESCRIBE {table_name}").df()
    print(schema.to_string(index=False))

    # Show sample rows
    print(f"\nSample data (first 3 rows):")
    sample = con.execute(f"SELECT * FROM {table_name} LIMIT 3").df()
    print(sample.to_string(index=False))

    # Show row count
    count = con.execute(f"SELECT COUNT(*) as count FROM {table_name}").fetchone()[0]
    print(f"\nTotal rows: {count:,}")


TABLE: clienti

Schema:
                        column_name column_type null  key default extra
                               Nome     VARCHAR  YES None    None  None
                            Cognome     VARCHAR  YES None    None  None
                                Età      BIGINT  YES None    None  None
                   Luogo di Nascita     VARCHAR  YES None    None  None
                 Luogo di Residenza     VARCHAR  YES None    None  None
                        Professione     VARCHAR  YES None    None  None
                            Reddito      BIGINT  YES None    None  None
                  Reddito Familiare      BIGINT  YES None    None  None
                       Numero Figli      BIGINT  YES None    None  None
         Anzianità con la Compagnia      BIGINT  YES None    None  None
                       Stato Civile     VARCHAR  YES None    None  None
          Numero Familiari a Carico      BIGINT  YES None    None  None
                    Reddito Stimato    

## Adding Primary and Foreign Keys

To add constraints, we need to recreate tables with explicit schemas.

In [9]:
# Step 1: Backup existing tables and recreate with constraints

# Drop existing tables if they exist (be careful!)
tables_to_recreate = [
    "clienti",
    "polizze",
    "sinistri",
    "reclami",
    "abitazioni",
    "interazioni_clienti",
]

# Create clienti with PRIMARY KEY
con.execute(
    """
    CREATE TABLE IF NOT EXISTS clienti_new AS 
    SELECT * FROM clienti
"""
)

con.execute("DROP TABLE IF EXISTS clienti")

con.execute(
    """
    CREATE TABLE clienti (
        Nome VARCHAR,
        Cognome VARCHAR,
        Età BIGINT,
        "Luogo di Nascita" VARCHAR,
        "Luogo di Residenza" VARCHAR,
        Professione VARCHAR,
        Reddito BIGINT,
        "Reddito Familiare" BIGINT,
        "Numero Figli" BIGINT,
        "Anzianità con la Compagnia" BIGINT,
        "Stato Civile" VARCHAR,
        "Numero Familiari a Carico" BIGINT,
        "Reddito Stimato" DOUBLE,
        "Patrimonio Finanziario Stimato" DOUBLE,
        "Patrimonio Reale Stimato" DOUBLE,
        "Consumi Stimati" DOUBLE,
        "Propensione Acquisto Prodotti Vita" DOUBLE,
        "Propensione Acquisto Prodotti Danni" DOUBLE,
        "Valore Immobiliare Medio" DOUBLE,
        "Probabilità Furti Stimata" DOUBLE,
        "Probabilità Rapine Stimata" DOUBLE,
        "Zona di Residenza" VARCHAR,
        codice_cliente BIGINT PRIMARY KEY,
        Agenzia VARCHAR,
        Latitudine DOUBLE,
        Longitudine DOUBLE,
        Num_Polizze BIGINT,
        Engagement_Score DOUBLE,
        Churn_Probability DOUBLE,
        CLV_Stimato BIGINT,
        Potenziale_Crescita DOUBLE,
        Reclami_Totali BIGINT,
        Satisfaction_Score DOUBLE,
        Data_Ultima_Visita DATE,
        Visite_Ultimo_Anno BIGINT,
        Cluster_Risposta VARCHAR
    )
"""
)

con.execute("INSERT INTO clienti SELECT * FROM clienti_new")
con.execute("DROP TABLE clienti_new")

print("✓ Created clienti with PRIMARY KEY on codice_cliente")

✓ Created clienti with PRIMARY KEY on codice_cliente


In [10]:
# Step 2: Recreate polizze with FOREIGN KEY to clienti
con.execute(
    """
    CREATE TABLE IF NOT EXISTS polizze_new AS 
    SELECT * FROM polizze
"""
)

con.execute("DROP TABLE IF EXISTS polizze")

con.execute(
    """
    CREATE TABLE polizze (
        "Unnamed: 0" DOUBLE,
        codice_cliente BIGINT,
        Prodotto VARCHAR,
        "Area di Bisogno" VARCHAR,
        "Data di Emissione" DATE,
        Premio_Ricorrente DOUBLE,
        Premio_Unico DOUBLE,
        Capitale_Rivalutato DOUBLE,
        Massimale DOUBLE,
        Stato_Polizza VARCHAR,
        Data_Scadenza DATE,
        Canale_Acquisizione VARCHAR,
        Commissione_Perc DOUBLE,
        Premio_Totale_Annuo DOUBLE,
        Commissione_Euro BIGINT,
        Costi_Operativi DOUBLE,
        Margine_Lordo BIGINT,
        Importo_Liquidato DOUBLE,
        Sinistri_Totali BIGINT,
        Loss_Ratio DOUBLE,
        FOREIGN KEY (codice_cliente) REFERENCES clienti(codice_cliente)
    )
"""
)

con.execute("INSERT INTO polizze SELECT * FROM polizze_new")
con.execute("DROP TABLE polizze_new")

print("✓ Created polizze with FOREIGN KEY to clienti")

✓ Created polizze with FOREIGN KEY to clienti


In [11]:
# Step 3: Add similar constraints to other tables
# Example for sinistri, reclami, abitazioni, interazioni_clienti

tables_with_fk = {
    "sinistri": """
        CREATE TABLE sinistri (
            codice_cliente BIGINT,
            Prodotto VARCHAR,
            "Area di Bisogno" VARCHAR,
            Sinistro VARCHAR,
            Data_Sinistro DATE,
            Importo_Liquidato BIGINT,
            Stato_Liquidazione VARCHAR,
            FOREIGN KEY (codice_cliente) REFERENCES clienti(codice_cliente)
        )
    """,
    "reclami": """
        CREATE TABLE reclami (
            codice_cliente BIGINT,
            Prodotto VARCHAR,
            "Area di Bisogno" VARCHAR,
            Reclami_e_info VARCHAR,
            FOREIGN KEY (codice_cliente) REFERENCES clienti(codice_cliente)
        )
    """,
    "abitazioni": """
        CREATE TABLE abitazioni (
            codice_cliente BIGINT,
            "Luogo di Residenza" VARCHAR,
            Indirizzo VARCHAR,
            Metratura BIGINT,
            Sistema_Allarme BOOLEAN,
            FOREIGN KEY (codice_cliente) REFERENCES clienti(codice_cliente)
        )
    """,
    "interazioni_clienti": """
        CREATE TABLE interazioni_clienti (
            codice_cliente BIGINT,
            Data_Interazione DATE,
            Tipo_Interazione VARCHAR,
            Motivo VARCHAR,
            Durata_Minuti DOUBLE,
            Esito VARCHAR,
            Note VARCHAR,
            Conversione BOOLEAN,
            FOREIGN KEY (codice_cliente) REFERENCES clienti(codice_cliente)
        )
    """,
}

for table_name, create_sql in tables_with_fk.items():
    # Backup
    con.execute(f"CREATE TABLE {table_name}_new AS SELECT * FROM {table_name}")
    # Drop old
    con.execute(f"DROP TABLE IF EXISTS {table_name}")
    # Create with constraints
    con.execute(create_sql)
    # Restore data
    con.execute(f"INSERT INTO {table_name} SELECT * FROM {table_name}_new")
    # Cleanup
    con.execute(f"DROP TABLE {table_name}_new")
    print(f"✓ Recreated {table_name} with FOREIGN KEY constraint")

print("\n✓ All constraints added successfully!")

✓ Recreated sinistri with FOREIGN KEY constraint
✓ Recreated reclami with FOREIGN KEY constraint
✓ Recreated abitazioni with FOREIGN KEY constraint
✓ Recreated interazioni_clienti with FOREIGN KEY constraint

✓ All constraints added successfully!


In [12]:
# Verify constraints are in place
print("Table constraints:\n")
for table in ["clienti", "polizze", "sinistri", "reclami", "abitazioni", "interazioni_clienti"]:
    constraints = con.execute(
        f"""
        SELECT 
            constraint_name, 
            constraint_type 
        FROM duckdb_constraints() 
        WHERE table_name = '{table}'
    """
    ).df()

    if not constraints.empty:
        print(f"{table}:")
        print(constraints.to_string(index=False))
        print()
    else:
        print(f"{table}: No constraints found")
        print()

Table constraints:

clienti:
                constraint_name constraint_type
    clienti_codice_cliente_pkey     PRIMARY KEY
clienti_codice_cliente_not_null        NOT NULL

polizze:
                           constraint_name constraint_type
polizze_codice_cliente_codice_cliente_fkey     FOREIGN KEY

sinistri:
                            constraint_name constraint_type
sinistri_codice_cliente_codice_cliente_fkey     FOREIGN KEY

reclami:
                           constraint_name constraint_type
reclami_codice_cliente_codice_cliente_fkey     FOREIGN KEY

abitazioni:
                              constraint_name constraint_type
abitazioni_codice_cliente_codice_cliente_fkey     FOREIGN KEY

interazioni_clienti:
                                       constraint_name constraint_type
interazioni_clienti_codice_cliente_codice_cliente_fkey     FOREIGN KEY



In [3]:
con.execute("CALL start_ui()")